In [ ]:
import tensorflow as tf
import cv2 as cv
import numpy as np
import os
from os.path import join

In [ ]:
def load_labels():
    fdir = "labels"
    dataset = []
    for file in os.listdir(fdir):
        if file.endswith(".jpg"):
            img = cv.imread(os.path.join(fdir,file), cv.IMREAD_COLOR)
            if int(file[0]) == 0:    
                dataset.append([img,np.array([0,1])])
            else:
                dataset.append([img,np.array([1,0])])
    return np.array(dataset)

In [ ]:
data = load_labels()
np.random.shuffle(data)

n_classes = 2
batch_size = 128
hm_epochs = 3

x = tf.placeholder('float',[24,24,3])
y = tf.placeholder('float')

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def maxpool2d(x):
    #                          size of window    moment of window
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

In [ ]:
def cnn_model(x):
    # 5 x 5, 1 input, produces 32 features
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,3,32])),
               'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
               'W_fc':tf.Variable(tf.random_normal([6*6*64, 1024])),
               'out':tf.Variable(tf.random_normal([1024, n_classes]))}
    
    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
              'b_conv2':tf.Variable(tf.random_normal([64])),
              'b_fc':tf.Variable(tf.random_normal([1024])),
              'out':tf.Variable(tf.random_normal([n_classes]))}
    
    x = tf.reshape(x, shape=[-1,24,24,3])
    #print("debug A")
    conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    #print("debug B")
    conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool2d(conv2)
    #print("debug C")
    fc = tf.reshape(conv2,[-1,6*6*64])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc']) + biases['b_fc'])
    #print("debug D")
    fc = tf.nn.dropout(fc,keep_rate)
    
    output = tf.matmul(fc, weights['out']) + biases['out']
    
    return output

In [ ]:
def cnn_train(x):
    prediction = cnn_model(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0;
            
            for i in range(len(data)):
                batch_x = data[i][0]
                batch_y = data[i][1]
                _, c = sess.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y})
                epoch_loss += c
            print('Epoch: ', epoch, ' completed out of: ', hm_epochs, ' loss: ', epoch_loss)
            
            
        ## Check how good model is
        correct = tf.equal (tf.argmax(prediction,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy: ',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

In [ ]:
cnn_train(x)